In [5]:
import numpy as np
from numpy import genfromtxt
from scipy.interpolate import interp1d
from bokeh.plotting import figure, output_notebook, show
from bokeh.layouts import row, column
from bokeh.io import push_notebook
from bokeh.models import Range1d, Label
output_notebook()

Loading BokehJS ...

In [71]:
import ipywidgets as widgets
from ipywidgets import FloatText, VBox, HBox, IntProgress

pump_wavelength_widget=widgets.FloatText(value=1064, disabled=False)
display(VBox([widgets.Label('Pump Wavelength (nm)'), pump_wavelength_widget]))

FWHM_widget=widgets.FloatText(value=1000, disabled=False)
Ep_widget=widgets.FloatText(value=1, disabled=False)
rep_rate_widget=widgets.FloatText(value=50, disabled=False)
display(HBox([VBox([widgets.Label('FWHM of Pump Pulse (psec)'), FWHM_widget]),VBox([widgets.Label('Pump Pulse Energy (nJ)'), Ep_widget]),
             VBox([widgets.Label('Pulse Repetition Rate (MHz)'), rep_rate_widget])]))

Aeff_widget=widgets.FloatText(value=25, disabled=False)
peak_gain_widget=widgets.FloatText(value=4, disabled=False)
pump_gain_widget=widgets.FloatText(value=0.04, disabled=False)
display(HBox([VBox([widgets.Label('Effective Area for Signal (micron^2)'), Aeff_widget]),VBox([widgets.Label('Peak Raman Gain (10^12 cm/W)'),
            peak_gain_widget]), VBox([widgets.Label('Pump Gain (1/m)'), pump_gain_widget])]))

seed_power_widget=widgets.FloatText(value=100, disabled=False)
fiber_length_widget=widgets.FloatText(value=300, disabled=False)
delta_L_widget=widgets.FloatText(value=40, disabled=False)
display(HBox([VBox([widgets.Label('Seed Power (mW)'), seed_power_widget]),VBox([widgets.Label('Fiber Length (m)'),
            fiber_length_widget]), VBox([widgets.Label('Numerical Step Size (cm)'), delta_L_widget])]))

In [70]:
wavelength = 1.07e-6  # units are meters
pump_wavelength=pump_wavelength_widget.value # pump wavelength in nm
FWHM = FWHM_widget.value # FWHM of pulse in psec
Ep = Ep_widget.value*1e-9 #pulse energy in Joules
rep_rate=rep_rate_widget.value*1e6 # pulse repetition rate in Hz
Aeff = Aeff_widget.value*1e-8 # Effective cross-sectional area for optical pulse in square microns
peak_gain=peak_gain_widget.value*1e-12 # Peak Raman Gain in cm/W
pump_gain=pump_gain_widget.value/100 # gain coefficient in inverse cm
seed_power=seed_power_widget.value/1000 # power in Watt
fiber_length=fiber_length_widget.value # fiber Length in meters
delta_L=delta_L_widget.value # numerical distance increment in cm

t=np.linspace(-2000,2000,101)
I0=(Ep/Aeff)*(np.sqrt(4*np.log(2)/np.pi)/FWHM)*1e12
pump=I0*np.exp(-4*np.log(2)*(t/FWHM)**2)
signal_wavelength=pump_wavelength+55
signal2_wavelength=signal_wavelength+55
hnu_signal2=1239.842/signal2_wavelength*1.6022e-19
pump=I0*np.exp(-4*np.log(2)*(t/FWHM)**2)
signal=np.zeros(101)+seed_power/Aeff
signal2=np.zeros(101)+hnu_signal2*20e12

pg=figure(plot_width=450, plot_height=400, y_axis_type="log")
pg.min_border_top = 10
pg.outline_line_width = 1
pg.outline_line_alpha = 1
pg.outline_line_color = "black"
pg.background_fill_color = "lightgrey"
pg.background_fill_alpha = 0.3
"""
label1=Label(x=10, y=235, x_units='screen', y_units='screen', text='Pump FWHM = {} psec'.format("%.0f" % FWHM),
             text_font_size='10pt', text_color='darkblue')
label2=Label(x=10, y=220, x_units='screen', y_units='screen', text='Pump Energy = {} J'.format("%.1e" % Ep),
             text_font_size='10pt', text_color='darkblue')
label3=Label(x=10, y=205, x_units='screen', y_units='screen', text='Seed Power = {} W'.format("%.1f" % seed_power),
             text_font_size='10pt', text_color='darkblue')
label4=Label(x=10, y=190, x_units='screen', y_units='screen', text='Fiber Length = {} m'.format("%.1f" % fiber_length),
             text_font_size='10pt', text_color='darkblue')
label5=Label(x=10, y=175, x_units='screen', y_units='screen', text='Aeff = {} cm^2'.format("%.1e" % Aeff),
             text_font_size='10pt', text_color='darkblue')
"""
label6=Label(x=10, y=300, x_units='screen', y_units='screen', text='Max 1st Stokes E',
             text_font_size='10pt', text_color='darkblue')
label7=Label(x=10, y=250, x_units='screen', y_units='screen', text='Max 1st Stokes I',
             text_font_size='10pt', text_color='darkblue')
label8=Label(x=10, y=200, x_units='screen', y_units='screen', text='Max Ave 1st Stokes Power',
             text_font_size='10pt', text_color='darkblue')
"""
pg.add_layout(label1)
pg.add_layout(label2)
pg.add_layout(label3)
pg.add_layout(label4)
pg.add_layout(label5)
"""
pg.add_layout(label6)
pg.add_layout(label7)
pg.add_layout(label8)

pg.x_range = Range1d(-2000,2000)
pg.y_range = Range1d(0.08,1e6)
pg.line(t, signal*Aeff, legend="CW Seed-1125nm", color='green', line_width=2, line_dash='dashed')
cp=pg.line(t,pump*Aeff, legend="Fundamental @ "+str(int(pump_wavelength))+" nm", color='blue', line_width=2)
c=pg.line(t, signal*Aeff, legend="1st Stokes @ 1125nm", color='green', line_width=2)
c2=pg.line(t, signal2*Aeff, legend="2nd Stokes @ 1180nm", color='red', line_width=2)
pg.legend.location = "top_right"
pg.xaxis.axis_label = 'Time (psec)'; pg.xaxis.axis_label_text_font_size = "12pt"
pg.xaxis.major_label_text_font_size = "12pt"
pg.yaxis.axis_label = 'Signal (Watt)'; pg.yaxis.axis_label_text_font_size = "12pt"
pg.yaxis.major_label_text_font_size = "12pt"

pc=figure(plot_width=450, plot_height=400, y_axis_type="log")
pc.outline_line_width = 1
pc.outline_line_alpha = 1
pc.outline_line_color = "black"
pc.background_fill_color = "lightgrey"
pc.background_fill_alpha = 0.3

pc.x_range = Range1d(0,fiber_length)
# pc.y_range = Range1d(-0.2*Ep,42*Ep)
# pc.y_range = Range1d(1e-24,2e-7)
pc.xaxis.axis_label = 'Distance Along Fiber (meter)'; pc.xaxis.axis_label_text_font_size = "12pt"
pc.xaxis.major_label_text_font_size = "12pt"
pc.yaxis.axis_label = 'Pulse Energy (Joule)'; pc.yaxis.axis_label_text_font_size = "12pt"
pc.yaxis.major_label_text_font_size = "12pt"

nh=show(row(pg,pc),notebook_handle='true')

import time
t0 = time.time()

rangen = int(fiber_length*1e2/delta_L)
P = IntProgress(min=0, max=rangen, description='Progress', bar_style='success'); display(P)
z=np.linspace(0,fiber_length,rangen+1)
Epz=np.zeros(rangen+1)
Epz[0]=Ep
Espz=np.zeros(rangen+1)
Es2z=np.zeros(rangen+1)
for i in range(rangen):
    delta_signal=pump*signal*peak_gain*delta_L
    signal=signal+delta_signal
    delta_signal2=signal*(signal2)*peak_gain*delta_L
    signal2=signal2+delta_signal2
    pump=(pump-delta_signal*pump_wavelength/signal_wavelength)*(1+pump_gain*delta_L)
    signal=signal-delta_signal2
    c.data_source.data['y']=signal*Aeff
    cp.data_source.data['y']=pump*Aeff
    c2.data_source.data['y']=signal2*Aeff
    
    Espz[i+1]=np.sum(signal*Aeff-0.1)*40e-12
    Es2z[i+1]=np.sum(signal2*Aeff)*40e-12
    Epz[i+1]=np.sum(pump*Aeff*40e-12)
    P.value=i
    if i % 20 == 0 or i == rangen-1:
        push_notebook(handle=nh)
        time.sleep(.200)

pc.line(z,Epz,color='blue',line_width=2,legend='Fundmenal @ '+str(int(pump_wavelength))+' nm')
pc.line(z,Espz,color='green',line_width=2,legend='1st Stokes - 1125 nm')
pc.line(z,Es2z,color='red',line_width=2,legend='2nd Stokes - 1180 nm')
pc.legend.location = "center_right"
push_notebook(handle=nh)

label6a=Label(x=10, y=280, x_units='screen', y_units='screen', text='= {} J'.format("%1.1e" % np.max(Espz)),
             text_font_size='10pt', text_color='darkblue')
pg.add_layout(label6a)

max_signal_irr=(np.max(Espz)/Aeff)*(np.sqrt(4*np.log(2)/np.pi)/FWHM)*1e12
label7a=Label(x=10, y=230, x_units='screen', y_units='screen', text='= {} W/cm^2'.format("%1.1e" % max_signal_irr),
             text_font_size='10pt', text_color='darkblue')
pg.add_layout(label7a)

max_ave_spower=np.max(Espz)*rep_rate
label8.text='Max Ave 1st Stokes Power = {} W'.format("%2.1f" % max_ave_spower)
push_notebook(handle=nh)

t1=time.time()
print('elapsed time=',t1-t0)

IntProgress(value=0, bar_style='success', description='Progress', max=750)

elapsed time= 17.568683862686157


In [ ]:
import pandas as pd
pd.options.display.max_rows = 999
Espz_rolled=np.roll(Espz,1)

df = pd.DataFrame({ 'Espz' : Espz, 'Espz_diff' : Espz-Espz_rolled,
                   'Epz' : Epz, 'Ep_diff' : Epz-np.roll(Epz,1) })
df